In [3]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB
import math

In [101]:
#Model has fixed size
def model(W,H):
######################DATA PREPARATION######################
#Parameters taken by the paper
    #env = gp.Env(empty=True)
    #env.setParam("OutputFlag",0)
    #env.start()


    model=gp.Model()
    FOC=1000
    MOB=20000
    OB=0.02
    OP=0.5
    OV=0.05
    AP=50000
    LP=4
    LR=0
    R=2
    RP=0.20
    rW=0.00123
    V=4
    if H!=1:
            CL=18000
    else:
            CL=model.addVar(name="CL")

#Parameters taken by the data (r1)
    N=16
    S=np.zeros(16, dtype=object)
    S[0]=[3,10,11]
    S[1]=[5,11]
    S[2]=[4,6,7]
    S[3]=[5,8]
    S[4]=[9,11]
    S[5]=[6,11]
    S[6]=[13,15]
    S[7]=[8,10]
    S[8]=[13,14,15]
    S[9]=[12,14]
    S[10]=[12,13]
    S[11]=[15]
    S[12]=[15]
    S[13]=[]
    S[14]=[]
    S[15]=[]
    M=np.zeros(16, dtype=object)
    for i in range(16):
        M[i]=[0,1,2]
    C=np.zeros([16,3],dtype=np.int64)
    C[0,0]=14
    C[0,1]=5
    C[0,2]=2
    C[1,0]=9
    C[1,1]=4
    C[1,2]=12
    C[2,0]=15
    C[2,1]=8
    C[2,2]=9
    C[3,0]=3
    C[3,1]=12
    C[3,2]=6
    C[4,0]=20
    C[4,1]=14
    C[4,2]=6
    C[5,0]=13
    C[5,1]=4
    C[5,2]=9
    C[6,0]=13
    C[6,1]=9
    C[6,2]=4
    C[7,0]=16
    C[7,1]=5
    C[7,2]=4
    C[8,0]=12
    C[8,1]=4
    C[8,2]=2
    C[9,0]=16
    C[9,1]=9
    C[9,2]=10
    C[10,0]=3
    C[10,1]=13
    C[10,2]=9
    C[11,0]=6
    C[11,1]=5
    C[11,2]=10
    C[12,0]=11
    C[12,1]=3
    C[12,2]=6
    C[13,0]=14
    C[13,1]=13
    C[13,2]=9
    C[14,0]=10
    C[14,1]=12
    C[14,2]=2
    C[15,0]=7
    C[15,1]=12
    C[15,2]=2
    MMF=np.zeros([16,3],dtype=np.int64)
    for i in range(N):
        for j in range(3):
            MMF[i,j]=C[i,0]/C[i,j]
    T=np.zeros([16,3],dtype=np.int64)
    T[0,0]=1
    T[0,1]=1
    T[0,2]=2
    T[1,0]=3
    T[1,1]=7
    T[1,2]=10
    T[2,0]=2
    T[2,1]=9
    T[2,2]=10
    T[3,0]=3
    T[3,1]=8
    T[3,2]=9
    T[4,0]=3
    T[4,1]=6
    T[4,2]=8
    T[5,0]=3
    T[5,1]=7
    T[5,2]=10
    T[6,0]=1
    T[6,1]=2
    T[6,2]=2
    T[7,0]=3
    T[7,1]=5
    T[7,2]=9
    T[8,0]=1
    T[8,1]=2
    T[8,2]=10
    T[9,0]=1
    T[9,1]=1
    T[9,2]=5
    T[10,0]=5
    T[10,1]=7
    T[10,2]=10
    T[11,0]=1
    T[11,1]=3
    T[11,2]=6
    T[12,0]=4
    T[12,1]=4
    T[12,2]=4
    T[13,0]=1
    T[13,1]=1
    T[13,2]=9
    T[14,0]=2
    T[14,1]=6
    T[14,2]=7
    T[15,0]=3
    T[15,1]=9
    T[15,2]=10
    
    DCsum=np.sum(C[:,0])#Equation 1 
    VOCsum=OV*DCsum# Equation 2
    FOCsum=DCsum*W# Equation 3
    PM=OP*(DCsum+VOCsum+FOCsum+MOB)#Equation 4
    BC=OB*(DCsum+VOCsum+FOCsum+MOB+PM)#Equation 5
    BP=DCsum+VOCsum+FOCsum+MOB+PM+BC#Equation 6
    MU=BP/DCsum#Equation 7

          
    
    #Equation from 7 to 20 are not in the model and equation from 21 to 22 are not used because of the fixed sizde
    x = model.addVars(N,N,W+1, vtype=GRB.BINARY, name="x") #equation 45
    DC = model.addVars(N,W+1, name="dc")
    EV = model.addVars(N,math.ceil(W/R)*R+1, name="ev")
    TE = model.addVars(W+1, name="te")
    I = model.addVars(math.ceil(W/R)*R+1, name="I")
    P = model.addVars(math.ceil(W/R)*R+LP+1, name="P")
    K=math.ceil(W/R)
    y = model.addVars(K+1, vtype=GRB.BINARY, name="y")#equation 47
    temp=model.addVar(vtype=GRB.BINARY, name="temp")
    Mg=model.addVar(name="Mg")
    Z=model.addVar(name="Z")
    IB = model.addVars(W+1, name="IB")
    B = model.addVars(W+1, name="B")
    IL = model.addVars(W+1, name="IL") 
    alpha= model.addVars(W+1, vtype=GRB.BINARY, name="alpha") #equation 48

    model.addConstrs(gp.quicksum(x[i,j,k] for j in M[i] for k in range(1,W+1))==1 for i in range(N))#equation 23
    model.addConstrs(gp.quicksum((k+T[i,j])*x[i,j,k] for j in M[i] for k in range(1,W+1))<gp.quicksum(k*x[q,j,k] for j in M[q] for k in range(1,W+1)) for q in S[i] for i in range(N)) #equation 24
    model.addConstrs(gp.quicksum((k+T[i,j])*x[i,j,k] for j in M[i] for k in range(1,W+1))<=W for i in range(N) if len(S[i])==0) #equation 25
    model.addConstrs(DC[i,k]==gp.quicksum(C[i,j]/T[i,j]*x[i,j,t] for t in range(max(1,k-T[i,j]),k+1) for j in M[i]) for i in range(N) for k in range(1,W+1)) #equation 26
    model.addConstrs(TE[k]==FOC+(1+OV)*gp.quicksum(DC[i,k] for i in range(N)) for k in range(1,W+1)) #equation 27
    model.addConstr(TE[0]==MOB+BC)
    model.addConstrs(EV[i,k]==gp.quicksum(MMF[i,j]*DC[i,k] for j in M[i] for t in range(max(1,k-T[i,j]),k+1)) for i in range(N) for k in range(1,W+1)) #equation 28
    model.addConstrs(EV[i,k]==0 for i in range(N) for k in range(W+1,math.ceil(W/R)*R+1)) #equation 28
    templist=[k for k in range(R,math.ceil(W/R)*R,R)]
    templist.append(math.ceil(W/R)*R)
    model.addConstrs(I[k]==MU*gp.quicksum(EV[i,t] for t in range(k-R+1,k+1) for i in range(N)) for k in templist) #equation 29
    model.addConstrs(I[k]==0 for k in set(range(math.ceil(W/R)*R))-set(templist))
    templist=[k for k in range(R+LP,math.ceil(W/R)*R,R)]
    templist.append(math.ceil(W/R)*R+LP)
    model.addConstrs(P[k]==RP*I[k-LP]-gp.quicksum(AP*y[n]/n for n in range(1,K+1)) for k in templist) #equation 30
    model.addConstrs(P[k]==0 for k in set(range(math.ceil(W/R)*R+LP+1))-set(templist)) 
    model.addConstr(R*Z>=W) #equation 31
    model.addConstr(R*Z<=W+R-1) #equation 32
    model.addConstr(gp.quicksum(n*y[n] for n in range(K+1))==Z) #equation 33-34
    model.addConstr(B[0]==AP-TE[0]) #equation 35
    model.addConstrs(B[k]==B[k-1]-TE[k]+P[k-1]-IB[k] for k in range(1,W+1)) #equation 36
    templist=[k for k in range(V,W,V)]
    templist.append(W)
    model.addConstrs(IB[k]==rW*gp.quicksum(IL[t] for t in range(max(1,k-V+1),k+1)) for k in templist) #equation 37
    model.addConstrs(B[k-1]-TE[k-1]<=(1-alpha[k])*Mg for k in range(1,W+1))#equation 38
    model.addConstrs(B[k-1]-TE[k-1]>=-alpha[k]*Mg for k in range(1,W+1))#equation 39
    model.addConstrs(IL[k]>=0  for k in range(1,W+1))
    model.addConstrs(IL[k]>=-(B[k-1]-TE[k])+(alpha[k]-1)*Mg for k in range(1,W+1)) #equation 41
    model.addConstrs(IL[k]<=-(B[k-1]-TE[k])+(1-alpha[k])*Mg for k in range(1,W+1)) #equation 42
    model.addConstrs(IL[k]<=alpha[k]*Mg  for k in range(1,W+1))#equation 43
    model.addConstrs(B[k]<=CL for k in range(1,W+1))
    model.addConstr(Z>=0) 
    model.addConstr(Z<=Mg)
 
    if H==1:
        model.setObjective(CL,GRB.MINIMIZE)    
    model.optimize()
    count=model.SolCount    

    if H==0:
        return model.SolCount

    if H==1:
        return CL.X
    

In [102]:
model(5,1)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 242 rows, 1800 columns and 1508 nonzeros
Model fingerprint: 0x050720f4
Model has 25 quadratic constraints
Variable types: 253 continuous, 1547 integer (1547 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+04]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 170 rows and 1556 columns
Presolve time: 0.00s
Presolved: 88 rows, 256 columns, 894 nonzeros
Presolved model has 8 SOS constraint(s)
Variable types: 29 continuous, 227 integer (198 binary)

Root relaxation: objective 4.065565e+03, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

    

4065.5649999999996